In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from models.layers import TransformerTimeSeriesForecast
import torch.optim as optim 
from torch import nn
from utils.auxiliary_fn import train_one_ep,evaluate_one_ep
from models.LSTM_FCN import LSTM_FCN_2_Atten
if torch.cuda.is_available():
    device=torch.device('cuda')
else :
    device=torch.device('cpu')

# split data

In [ ]:
from utils.train_val_test import h_d_train,h_d_val,h_d_test,get_h_f_dataloader,get_data

span_len=60
pre_len=1
batch_size=32

train_dataset=h_d_train(cut_len=span_len,slide_win_size=pre_len,transform=False)
val_dataset=h_d_val(cut_len=span_len,slide_win_size=pre_len)
test_dataset=h_d_test(cut_len=span_len,slide_win_size=pre_len)


train_dataloader=get_h_f_dataloader(train_dataset,batch_size,0)
val_dataloader=get_h_f_dataloader(val_dataset,batch_size,0)
test_dataloader=get_h_f_dataloader(test_dataset,batch_size,0)

x,y=next(iter(train_dataloader))
# print(x.shape)
# print(y.shape)

print(x[30,29,:])
print(y[29])

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

class TimeSeriesDataset(Dataset):
    def __init__(self, data, targets, sequence_length):
        self.data = data
        self.targets = targets
        self.sequence_length = sequence_length

    def __getitem__(self, index):
        # Select the time series starting from index
        time_series = self.data[index:index+self.sequence_length]

        # Return the time series and the target value after the series
        return time_series, self.targets[index+self.sequence_length-1]

    def __len__(self):
        # Return the number of time series in the dataset
        return len(self.data) - self.sequence_length + 1

# Generate some random time series data
time_series_data = np.random.randn(1000, 2)

# Generate some random target values
targets = np.random.randint(0, 10, size=(1000,))

# Create a time series dataset
dataset = TimeSeriesDataset(torch.tensor(time_series_data,dtype=torch.float32), torch.tensor(targets,dtype=torch.float32), sequence_length=60)

# Create a dataloader for the dataset
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
x,y=next(iter(dataloader))
print(x.shape)
print(y.shape)

In [ ]:
import torch
import torch.nn as nn

class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=4):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

        self.fc1 = nn.Linear(hidden_dim, output_dim)

        # self.fc2= nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc1(out[:, -1, :])

        return out

'''定义模型和优化器'''
# model = TransformerTimeSeriesForecast(
#     feature_dim=2,
#     pre_len=pre_len,
#     num_heads=2,
#     hidden_dim=64,
#     num_layers=10,
#     data_len=span_len
#     )

model = LSTMNet(input_dim=2, hidden_dim=32, output_dim=1, num_layers=1).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.0015)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.00015,betas=(0.9,0.999))
# 定义损失函数
# criterion = nn.MSELoss()
from utils.loss_fn.TildeQ_loss import TildeQLoss
from utils.loss_fn.standard_loss import mase_loss,huber_loss,log_cosh_loss,pinball_loss,wmape_loss,smape_loss
criterion=mase_loss

# 训练循环
num_epochs=20000
epoch_loss=1000000000
# 训练循环
for epoch in range(num_epochs):
    running_loss = 0.0  # 定义一个变量用于累计当前 epoch 的总损失
    for i, data in enumerate(train_dataloader):
        inputs, targets = data
        inputs, targets = inputs.to(device), targets.to(device)

        # 前向传播
        outputs = model(inputs)

        # 计算损失
        loss = criterion(outputs, targets)
        
        # 反向传播，更新参数
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 累计当前 epoch 的总损失
        running_loss += loss.item()

        # # 打印训练状态
        # if (i + 1) % 10 == 0:
        #     print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, i + 1, len(train_dataloader), loss.item()))

    # 打印当前 epoch 的平均损失
    if epoch_loss>(running_loss / len(train_dataloader.dataset)):
        epoch_loss = running_loss / len(train_dataloader.dataset)
        print('Epoch [{}/{}], Average Loss: {:.4f}'.format(epoch + 1, num_epochs, epoch_loss))

# try transformer

In [ ]:
# model = TransformerTimeSeriesForecast(
#     feature_dim=7,
#     pre_len=pre_len,
#     num_heads=8,
#     hidden_dim=256,
#     num_layers=10,
#     data_len=span_len
#     )

model=LSTM_FCN_2_Atten(
    data_len=span_len,
    pre_len=pre_len,
    input_dim=1,
    hidden_dim=8,
    num_layers=8
)

learning_rate=0.0015
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

# Define the loss function
# loss_fn = nn.SmoothL1Loss()  ## optional_1

from utils.loss_fn.soft_dtw import SoftDTW
# Create the "criterion" object
loss_fn = SoftDTW(use_cuda=False, gamma=0.1)


# Create the data loader
if torch.cuda.is_available():
    device=torch.device('cuda')
else :
    device=torch.device('cpu')
# Train the model

num_epochs = 100


In [ ]:
def train_and_evaluate(model, criterion, optimizer, train_loader, evaluate_loader,val_loader,train_fn,test_fn,device,num_epochs):
    """
    Trains and evaluates a PyTorch model for time series forecasting.
    
    Args:
        model: The PyTorch model to train and evaluate.
        criterion: The loss function to use for training and evaluation.
        optimizer: The optimizer to use for training.
        train_loader (torch.utils.data.DataLoader): The data loader to use for training.
        val_loader (torch.utils.data.DataLoader): The data loader to use for evaluation.
        num_epochs (int): The number of epochs to train the model for.
    """
    for epoch in range(num_epochs):
        avg_train_loss=train_fn(optimizer, criterion, train_loader, model,device)
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}")
        
        # Evaluation loop
        model.eval()
        val_loss = test_fn(criterion, val_loader, model,device)
        print(f"Epoch [{epoch+1}/{num_epochs}], Val Loss: {val_loss:.4f}")
    
    # Final evaluation on validation set
    model.eval()
    val_loss = test_fn(criterion, evaluate_loader, model,device)
    print(f"Final Val Loss: {val_loss:.4f}")

In [ ]:
# train_and_evaluate(
#     model=model,
#     criterion=loss_fn,
#     optimizer=optimizer,
#     train_loader=train_dataloader,
#     val_loader=val_dataloader,
#     evaluate_loader=test_dataloader,
#     train_fn=train_one_ep,
#     test_fn=evaluate_one_ep,
#     device=device,
#     num_epochs=num_epochs)